In [1]:
import yaml

In [8]:
from collections import defaultdict

In [55]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf_all = yaml.safe_load(config_file)

In [56]:
conf_all

{'connection': {'host': '172.22.0.1',
  'db': 'dv2',
  'port': 5432,
  'user': 'postgres',
  'pw': 'postgres',
  'schemas': {'stage': 'stage', 'edwh': 'vault'}},
 'rv': {'hubs': {'hub_tax_bundle': {'hk': 'hk_tax_bundle',
    'bk': 'tax_bundle_id',
    'overwrite': False,
    'src': {'v_report_taxation_bundle': {'bk': 'tax_bundle_id',
      'bkeycode': 'default',
      'tenant': 'default'},
     'v_report_ziffer': {'bk': 'tax_bundle_id'}}}}}}

In [57]:
def defaultify(d):
    if isinstance(d, dict):
        return defaultdict(lambda: None, {k: defaultify(v) for k, v in d.items()})
    elif isinstance(d, list):
        return [defaultify(e) for e in d]
    else:
        return d

In [58]:
dconf=defaultify(conf_all)

In [83]:
dconf.

{'connection': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
             {'host': '172.22.0.1',
              'db': 'dv2',
              'port': 5432,
              'user': 'postgres',
              'pw': 'postgres',
              'schemas': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
                          {'stage': 'stage', 'edwh': 'vault'})}),
 'rv': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
             {'hubs': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
                          {'hub_tax_bundle': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
                                       {'hk': 'hk_tax_bundle',
                                        'bk': 'tax_bundle_id',
                                        'overwrite': False,
                                        'src': defaultdict(<function __main__.defaultify.<locals>.<lambda>()>,
                                              

# hubs


## loader

In [79]:
sql= open('./dags/sql/hub_loader.sql','r').read()

In [66]:
sql

"with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (\n\tselect md5(concat_ws('|',\n\t\t\t   {tenant},{bkeycode},\n\t\t\t   trim(src.{bk_src}::varchar(256),\n\t)) \n\t, trim(src.{bk_src}::varchar(256))\n\t, current_timestamp \n\t, coalesce({appts::timestamp},current_timestamp))\n\t, {rec_src}\n\t, {tenant}\n\t, {bkeycode}\n\t, {taskid}\t\n\tfrom {schema_src}.{table_src} src\n)\ninsert into {schema_tgt}.{table_tgt} (\n\t\t{hk_tgt}\n\t  , {bk_tgt}\n\t  , dv_loadts\n\t  , dv_appts\n\t  , dv_recsrc\n\t  , dv_tenant\n\t  , dv_bkeycode\n\t  , dv_taskid\n\t)\nselect distinct \n\thk\n  , bk\n  , dv_loadts\n  , dv_appts\n  , dv_recsrc\n  , dv_tenant\n  , dv_bkeycode\n  , dv_taskid\nfrom cte\nwhere not exists \n(\n\tselect 1 from {schema_tgt}.{table_tgt} tgt\n\twhere cte.hk = tgt.{hk_tgt}\n);"

In [80]:
conf=dconf['rv']

appts=None
sql_concat=''
hub='hub_tax_bundle'
schema_stage='stage'
schema_edwh='vault'

for cur_src,cur_conf in conf['hubs'][hub]['src'].items():

    params={'tenant':cur_conf['tenant'] if cur_conf['tenant'] else "default",
            'bkeycode':cur_conf['bkeycode'] if cur_conf['bkeycode'] else "default",
            'taskid': '1234',
            'appts':'appts::timestamp' if appts else 'current_timestamp',
            'rec_src':cur_src,
            'bk_src':cur_conf['bk'],
            'schema_src':schema_stage,
            'table_src':cur_src,
            'bk_tgt':conf['hubs'][hub]['bk'],
            'hk_tgt':conf['hubs'][hub]['hk'],
            'schema_tgt':schema_edwh,
            'table_tgt':hub
    }

    #print(sql.format(**params))
    sql_concat += '\n'+sql.format(**params)
    
print(sql_concat)    


with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (
	select md5(concat_ws('|',
			   default,default,
			   trim(src.tax_bundle_id::varchar(256),
	)) 
	, trim(src.tax_bundle_id::varchar(256))
	, current_timestamp 
	, coalesce(current_timestamp::timestamp,current_timestamp)
	, v_report_taxation_bundle
	, "default"
	, "default"
	, 1234	
	from stage.v_report_taxation_bundle src
)
insert into vault.hub_tax_bundle (
		hk_tax_bundle
	  , tax_bundle_id
	  , dv_loadts
	  , dv_appts
	  , dv_recsrc
	  , dv_tenant
	  , dv_bkeycode
	  , dv_taskid
	)
select distinct 
	hk
  , bk
  , dv_loadts
  , dv_appts
  , dv_recsrc
  , dv_tenant
  , dv_bkeycode
  , dv_taskid
from cte
where not exists 
(
	select 1 from vault.hub_tax_bundle tgt
	where cte.hk = tgt.hk_tax_bundle
);
with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (
	select md5(concat_ws('|',
			   default,default,
			   trim(src.tax_bundle_id::varchar(256),
	)) 
	, trim(src.tax_bundle_

# links

In [108]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf = yaml.safe_load(config_file)

In [109]:
conf

{'connection': {'host': '172.22.0.1',
  'db': 'dv2',
  'port': 5432,
  'user': 'postgres',
  'pw': 'postgres',
  'schemas': {'stage': 'stage', 'edwh': 'vault'}},
 'rv': {'hubs': {'hub_tax_bundle': {'hk': 'hk_tax_bundle',
    'bk': 'tax_bundle_id',
    'overwrite': False,
    'src': {'v_report_taxation_bundle': {'bk': 'tax_bundle_id',
      'bkeycode': 'default',
      'tenant': 'default'},
     'v_report_ziffer': {'bk': 'tax_bundle_id'}}},
   'hub_ziffer': {'hk': 'hk_ziffer',
    'bk': 'ziffer',
    'overwrite': False,
    'src': {'v_report_dict_ziffer': {'bk': 'ziffer'}}}},
  'links': {'lnk_taxbundle_ziffer': {'hk': 'hk_l_taxbundle_ziffer',
    'overwrite': False,
    'src': {'v_report_ziffer': {'bkeycode': 'default',
      'tenant': 'default',
      'bks': ['tax_bundle_id', 'ziffer']},
     'v_report_taxation_bundle': {'bks': ['tax_bundle_id', 'ziffer'],
      'bkeycode': 'default',
      'tenant': 'default'}}}}}}

In [86]:
dconf=defaultify(conf)

## link creator

In [87]:
sql= open('./dags/sql/link_creator.sql','r').read()

In [88]:
sql

'CREATE TABLE {schema}.{link} (\n\t{hk} varchar(256) NOT NULL,\n\t{bk_list}\n\t-- technical fields\n\tDV_LOADTS timestamp NULL,\n\tDV_APPTS timestamp NULL,\n\tDV_RECSRC varchar(256) NULL,\n\tDV_TENANT varchar(256) NULL,\n\tDV_BKEYCODE varchar(256) NULL,\n\tDV_TASKID varchar(256) NULL,\n    CONSTRAINT {link}_pkey PRIMARY KEY ({hk})\n);'

In [91]:
link='lnk_taxbundle_ziffer'

In [110]:
for cur_src, cur_conf in conf['rv']['links'][link]['src'].items():
    
    link_bks_dict=cur_conf['bks']    
    print(link_bks_dict)


['tax_bundle_id', 'ziffer']
['tax_bundle_id', 'ziffer']


In [53]:
link_bks_dict

{'tax_bundle_id': {'type': 'int'}, 'ziffer': {'type': 'int'}}

In [83]:
link_bks=''
for key,val in link_bks_dict.items():
    #print(key,val)
    link_bks += key + ' ' + val['type'] +' NOT NULL ,'    
link_bks

'tax_bundle_id int NOT NULL ,ziffer int NOT NULL ,'

In [84]:
params = {
    'schema':conf['connection']['schemas']['edwh'],
    'link':link,
    'hk':conf['links'][link]['hk'],
    'bk_list':link_bks
}

In [85]:
sql.format(**params)

'CREATE TABLE vault.l_taxbundle_ziffer (\n\thk_l_taxbundle_ziffer varchar(256) NOT NULL,\n    load_dts timestamp NULL,     \n    tax_bundle_id int NOT NULL ,ziffer int NOT NULL ,\n    CONSTRAINT l_taxbundle_ziffer_pkey PRIMARY KEY (hk_l_taxbundle_ziffer)\n);'

## link loader

In [131]:
sql= open('./dags/sql/link_loader.sql','r').read()

In [90]:
link='l_taxbundle_ziffer'

In [91]:
link_bks_dict=conf['links'][link]['src']['bks']

In [133]:
bk_concat_str="CONCAT_WS('|'"
bk_list=''
bk_list2=''
for key,val in link_bks_dict.items():    
    bk_list += 'src.'+key+','    
    bk_list2 += key+','    
    bk_concat_str+=f",trim(src.{key}::varchar(256))"

bk_list2=bk_list2[:-1] 
bk_list=bk_list[:-1]

In [134]:
bk_list

'src.tax_bundle_id,src.ziffer'

In [121]:
bk_list2

'tax_bundle_id,ziffer'

In [126]:
params = {
    'bk_concat_str':bk_concat_str,
    'bk_list':bk_list,
    'bk_list2':bk_list2,
    'schema_src':conf['connection']['schemas']['stage'],
    'table_src':conf['links'][link]['src']['table'],
    'schema_tgt':conf['connection']['schemas']['edwh'],
    'table_tgt':link,
    'hk_tgt': conf['links'][link]['hk']
}

In [132]:
print(sql.format(**params))

with cte (hk,load_dts,rec_src,bk) as (
	select md5(CONCAT_WS('|',trim(src.tax_bundle_id::varchar(256)),trim(src.ziffer::varchar(256))) 
	,  current_timestamp 
	, src.tax_bundle_id,src.ziffer,
	from stage.v_report_ziffer src
)
insert into vault.l_taxbundle_ziffer (hk_l_taxbundle_ziffer,load_dts,tax_bundle_id,ziffer)
select distinct hk
  , load_dts
  , tax_bundle_id,ziffer
from cte
where not exists 
(
	select 1 from vault.l_taxbundle_ziffer tgt
	where cte.hk = tgt.hk_l_taxbundle_ziffer
);
